## 2.9　结构化数据：NumPy的结构化数组

大多数时候，我们的数据可以通过一个异构类型值组成的数组表示，但有时却并非如此。

本节介绍NumPy 的结构化数组和记录数组，它们为复合的、异构的数据提供了非常有效的存储。

尽管这里列举的模式对于简单的操作非常有用，但是这些场景通常也可以用Pandas 的DataFrame 来实现（将在第三章详细介绍）。

假定现在有关于一些人的分类数据（如姓名、年龄和体重），我们需要存储这些数据用于Python 项目，那么一种可行的方法是将它们存在三个单独的数组中：

In [4]:
import numpy as np

In [1]:
name = ['Alice', 'Bob', 'Cathy', 'Doug']
age = [25, 45, 37, 19]
weight = [55.0, 85.5, 68.0, 61.5]

但是这种方法有点笨，因为并没有任何信息告诉我们这三个数组是相关联的。如果可以用一种单一结构来存储所有的数据，那么看起来会更自然。

NumPy 可以用结构化数组实现这种存储，这些结构化数组是复合数据类型的。
<br>前面介绍过，利用以下表达式可以生成一个简单的数组：

In [5]:
# 使用复合数据结构的结构化数组
data = np.zeros(4, dtype={'names':('name', 'age', 'weight'), 
                          'formats':('U10', 'i4', 'f8')})
print(data.dtype)

[('name', '<U10'), ('age', '<i4'), ('weight', '<f8')]


这里U10 表示“长度不超过10 的Unicode 字符串”，i4 表示“4 字节（即32 比特）整型”，f8 表示“8 字节（即64 比特）浮点型”。

现在生成了一个空的数组容器，可以将列表数据放入数组中：

In [7]:
data['name'] = name
data['age'] = age
data['weight'] = weight
print(data)

[('Alice', 25, 55. ) ('Bob', 45, 85.5) ('Cathy', 37, 68. )
 ('Doug', 19, 61.5)]


正如我们希望的，所有的数据被安排在一个内存块中。

结构化数组的方便之处在于，你可以通过索引或名称查看相应的值：

In [12]:
# 获取所有名字
data['name']

array(['Alice', 'Bob', 'Cathy', 'Doug'], dtype='<U10')

In [13]:
# 获取数据第一行
data[0]

('Alice', 25, 55.)

In [14]:
# 获取最后一行的名字
data[-1]['name']

'Doug'

利用布尔掩码，还可以做一些更复杂的操作，如按照年龄进行筛选：

In [15]:
# 获取年龄小于30岁的人的名字
data[data['age'] < 30]['name']

array(['Alice', 'Doug'], dtype='<U10')

请注意，如果你希望实现比上面更复杂的操作，那么你应该考虑使用Pandas 包。Pandas 提供了一个DataFrame 对象，该结构是
构建于NumPy 数组之上的，提供了很多有用的数据操作功能，其中有些与前面介绍的类似，当然也有更多没提过并且非常实用的功能。

### 2.9.1　生成结构化数组

结构化数组的数据类型有多种制定方式。此前我们看过了采用字典的方法：

In [ ]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':('U10', 'i4', 'f8')})

为了简明起见，数值数据类型可以用Python 类型或NumPy 的dtype 类型指定：

In [ ]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':((np.str_, 10), int, np.float32)})

复合类型也可以是元组列表：

In [ ]:
np.dtype([('name', 'S10'), ('age', 'i4'), ('weight', 'f8')])

如果类型的名称对你来说并不重要，那你可以仅仅用一个字符串来指定它。在该字符串中数据类型用逗号分隔：

In [ ]:
np.dtype('S10,i4,f8')

简写的字符串格式的代码可能看起来令人困惑，但是它们其实基于非常简单的规则。第一个（可选）字符是< 或者>，分别表示“低字节序”（little endian）和“高字节序”（bid endian），表示字节（bytes）类型的数据在内存中存放顺序的习惯用法。后一个字符指定的是数据的类型：字符、字节、整型、浮点型，等等（如表2-4 所示）。最后一个字符表该对象的字节大小

表2-4：NumPy的数据类型

|NumPy数据类型符号|描述|示例|
|:|:|:|
|'b'| 字节型|np.dtype('b')|
|'i'| 有符号整型|np.dtype('i4') == np.int32|
|'u'| 无符号整型|np.dtype('u1') == np.uint8|
|'f'| 浮点型|np.dtype('f8') == np.int64|
|'c'| 复数浮点型|np.dtype('c16') == np.complex128|
|'S'、'a'| 字符串|np.dtype('S5')|
|'U'| Unicode 编码字符串|np.dtype('U') == np.str_|
|'V'| 原生数据，raw data（空，void）| np.dtype('V') == np.void|

### 2.9.2　更高级的复合类型

NumPy 中也可以定义更高级的复合数据类型。

例如，你可以创建一种类型，其中每个元素都包含一个数组或矩阵。

我们会创建一个数据类型，该数据类型用mat 组件包含一个3×3的浮点矩阵：

In [17]:
tp = np.dtype([('id', 'i8'), ('mat', 'f8', (3, 3))])
X = np.zeros(1, dtype=tp)
print(X[0])
print(X['mat'][0])

(0, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


现在X 数组的每个元素都包含一个id 和一个3×3 的矩阵。

为什么我们宁愿用这种方法存储数据，也不用简单的多维数组，或者Python 字典呢？

原因是NumPy 的dtype 直接映射到C 结构的定义，因此包含数组内容的缓存可以直接在C 程序中使用。

如果你想写一个Python 接口与一个遗留的C 语言或Fortran 库交互，从而操作结构化数据，你将会发现结构化数组非常有用！

### 2.9.3　记录数组：结构化数组的扭转

NumPy 还提供了np.recarray 类。它和前面介绍的结构化数组几乎相同，但是它有一个独特的特征：**域可以像属性一样获取，而不是像字典的键那样获取**。

前面的例子通过以下代码获取年龄：

In [19]:
data['age']

array([25, 45, 37, 19])

如果将这些数据当作一个记录数组，我们可以用很少的按键来获取这个结果：

In [18]:
data_rec = data.view(np.recarray)
data_rec.age

array([25, 45, 37, 19])

记录数组的不好的地方在于，即使使用同样的语法，在获取域时也会有一些额外的开销，如以下示例所示：

In [21]:
%timeit data['age']
%timeit data_rec['age']
%timeit data_rec.age

242 ns ± 20.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
6.88 µs ± 241 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
10.2 µs ± 997 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


是否值得为更简便的标记方式花费额外的开销，这将取决于你的实际应用。